## Spark ML - badanie oszustw z kartami kredytowymi

Źródło danych: https://www.kaggle.com/mlg-ulb/creditcardfraud

#### Wczytanie danych

In [108]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import DoubleType

In [113]:
from pyspark import SparkConf

conf = new SparkConf()
             .setMaster("local[12]")
             .setAppName("CountingSheep")
        
sc = new SparkContext(conf)

SyntaxError: invalid syntax (<ipython-input-113-14451c5723d6>, line 3)

In [53]:
spark = SparkSession.builder.master('local').appName('MiNADZD').getOrCreate()

df = spark.read.csv('./dataset/finalDataset.csv', header=True, inferSchema=True)

df = df.drop('ind')
df = df.drop('_C0')
df = df.drop('id')

for feature in df.columns:
    df = df.withColumn(feature, df[feature].cast(DoubleType()))

df

DataFrame[Customer Type: double, Age: double, Type of Travel: double, Class: double, Flight Distance: double, Inflight wifi service: double, Departure/Arrival time convenient: double, Ease of Online booking: double, Gate location: double, Food and drink: double, Online boarding: double, Seat comfort: double, Inflight entertainment: double, On-board service: double, Leg room service: double, Baggage handling: double, Checkin service: double, Inflight service: double, Cleanliness: double, Departure Delay in Minutes: double, Arrival Delay in Minutes: double, satisfaction: double, Male: double, Female: double]

Rozmiar danych po wczytaniu zbioru:

In [54]:
'Rozmiar danych: ' + str(df.count()) + ' rows x ' + str(len(df.columns)) + ' columns.'

'Rozmiar danych: 129880 rows x 24 columns.'

In [55]:
df.show(5)

+------------------+-------------------+------------------+-------------------+-------------------+---------------------+---------------------------------+----------------------+-------------------+-------------------+-------------------+-------------------+----------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------+------------------------+------------+-------------------+-------------------+
|     Customer Type|                Age|    Type of Travel|              Class|    Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|      Gate location|     Food and drink|    Online boarding|       Seat comfort|Inflight entertainment|   On-board service|   Leg room service|   Baggage handling|    Checkin service|   Inflight service|        Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|satisfaction|               Male|       

Przeniesienie wszystkich cech do jednej kolumny w wektorze

In [56]:
inputColumns = []
for feature in df.columns:
    if feature != 'satisfaction':
        inputColumns.append(feature)

assembler = VectorAssembler(inputCols=inputColumns, outputCol='features')
df = assembler.transform(df)

In [58]:
df = df.select('satisfaction', 'features')
df

DataFrame[satisfaction: double, features: vector]

Normalizacja

In [59]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
liNormData = normalizer.transform(df)
liNormData.select('normFeatures').show(5)

+--------------------+
|        normFeatures|
+--------------------+
|[0.02314763577377...|
|[0.02649627212010...|
|[-0.0918628977331...|
|[0.01829704167050...|
|[0.02757044123145...|
+--------------------+
only showing top 5 rows



Podział na dane uczące i testowe

In [60]:
(trainingNormData, testNormData) = liNormData.randomSplit([0.8,0.2])

### RandomForestClassifier - trenowanie & testowanie 

In [98]:
algo = RandomForestClassifier(featuresCol='normFeatures', labelCol='satisfaction')
model = algo.fit(trainingNormData)

In [99]:
predictions = model.transform(testNormData)

Wyświetlenie pierwszych 20 wyników klasyfikacji:

In [100]:
predictions.select(['satisfaction','prediction', 'probability']).show(5)

+------------+----------+--------------------+
|satisfaction|prediction|         probability|
+------------+----------+--------------------+
|         0.0|       0.0|[0.90608539504542...|
|         0.0|       0.0|[0.92742229040276...|
|         0.0|       0.0|[0.86874318195797...|
|         0.0|       0.0|[0.89161591952192...|
|         0.0|       0.0|[0.86114690813448...|
+------------+----------+--------------------+
only showing top 5 rows



### Ocena jakości klasyfikatora

Dokładność i błąd klasyfikacji

In [101]:
evaluator = MulticlassClassificationEvaluator(labelCol='satisfaction', predictionCol="prediction", metricName='accuracy')
evaluator.evaluate(predictions)

0.9062380988651078

F1-score

In [102]:
evaluator = MulticlassClassificationEvaluator(labelCol='satisfaction', predictionCol="prediction", metricName='f1')
evaluator.evaluate(predictions)

0.9058882223725465

AUC

In [103]:
evaluator = BinaryClassificationEvaluator(labelCol='satisfaction', rawPredictionCol='rawPrediction')
evaluator.evaluate(predictions)

0.9625537861117727

Confusion Matrix

In [106]:
rdd = predictions.select('prediction','satisfaction').rdd

In [107]:
mm = MulticlassMetrics(rdd)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 395.0 failed 1 times, most recent failure: Lost task 0.0 in stage 395.0 (TID 379) (MSI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.

In [104]:
metrics = MulticlassMetrics(predictions.select('prediction','satisfaction').rdd)
metrics.confusionMatrix()
metrics

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 394.0 failed 1 times, most recent failure: Lost task 0.0 in stage 394.0 (TID 378) (MSI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.

### LogisticRegression - trenowanie & testowanie 

In [84]:
algo = LogisticRegression(featuresCol='normFeatures', labelCol='satisfaction')
model = algo.fit(trainingNormData)

In [85]:
predictions = model.transform(testNormData)

Wyświetlenie pierwszych 20 wyników klasyfikacji:

In [86]:
predictions.select(['satisfaction','prediction']).show(5)

+------------+----------+
|satisfaction|prediction|
+------------+----------+
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
+------------+----------+
only showing top 20 rows



### Ocena jakości klasyfikatora

Dokładność i błąd klasyfikacji

In [87]:
evaluator = MulticlassClassificationEvaluator(labelCol='satisfaction', predictionCol="prediction", metricName='accuracy')
evaluator.evaluate(predictions)

0.8683068017366136

F1-score

In [88]:
evaluator = MulticlassClassificationEvaluator(labelCol='satisfaction', predictionCol="prediction", metricName='f1')
evaluator.evaluate(predictions)

0.8679168483037503

AUC

In [89]:
evaluator = BinaryClassificationEvaluator(labelCol='satisfaction', rawPredictionCol='rawPrediction')
evaluator.evaluate(predictions)

0.9225455589396441

Confusion Matrix

In [90]:
metrics = MulticlassMetrics(predictions.select('prediction','satisfaction').rdd)
metrics.confusionMatrix()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 144.0 failed 1 times, most recent failure: Lost task 0.0 in stage 144.0 (TID 136) (MSI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.

### Gradient-Boosted Tree Classfier - trenowanie & testowanie 

In [91]:
algo = GBTClassifier(featuresCol='normFeatures', labelCol='satisfaction')
model = algo.fit(trainingNormData)

In [92]:
predictions = model.transform(testNormData)

Wyświetlenie pierwszych 20 wyników klasyfikacji:

In [93]:
predictions.select(['satisfaction','prediction']).show()

+------------+----------+
|satisfaction|prediction|
+------------+----------+
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
+------------+----------+
only showing top 20 rows



### Ocena jakości klasyfikatora

Dokładność i błąd klasyfikacji

In [94]:
evaluator = MulticlassClassificationEvaluator(labelCol='satisfaction', predictionCol="prediction", metricName='accuracy')
evaluator.evaluate(predictions)

0.9343438190265824

F1-score

In [95]:
evaluator = MulticlassClassificationEvaluator(labelCol='satisfaction', predictionCol="prediction", metricName='f1')
evaluator.evaluate(predictions)

0.9342557700363305

AUC

In [96]:
evaluator = BinaryClassificationEvaluator(labelCol='satisfaction', rawPredictionCol='rawPrediction')
evaluator.evaluate(predictions)

0.980607703730046

Confusion Matrix

In [97]:
metrics = MulticlassMetrics(predictions.select('prediction','satisfaction').rdd)
metrics.confusionMatrix()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 363.0 failed 1 times, most recent failure: Lost task 0.0 in stage 363.0 (TID 351) (MSI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.